In [1]:
import sys
# Add the ptdraft folder path to the sys.path list
sys.path.append('/src')

import numpy as np
import pandas as pd
from HSIC.HSIC import HSIC_pval
import time

In [2]:
WORDS_ORIGIN = "/src/data/procesados/clean/TWITTER_COLOMBIA_RELATIVE_FREQ_CLEAN.csv"
CITIES_ORIGIN = "/src/data/procesados/geo/filter_cities_coordinates.csv"
HSIC_ORIGIN = "/src/data/procesados/HSIC/20MIL/HSIC_SIN_PVALUE_REL_FREQ_NOLIN_20MIL.csv"
DESTINO = "/src/data/procesados/HSIC/HSIC_PVALUE.csv"

In [3]:
def escribir_cabezote_archivo(destino, fieldnames):
    with open(destino, 'w') as csvfile:
        writer = fieldnames.to_csv(csvfile,sep="\t",header=fieldnames.columns,decimal=",")
        
def escribir_datos_archivo(destino, values):
    with open(destino, 'a') as csvfile:
        writer = values.to_csv(csvfile,sep="\t",header=False,decimal=",") 

In [4]:
def hsicpvalue_to_file(hsic_origin, words_origin, cities_origin, destino, desde, hasta):
    
    # Cargar filtrados hsic
    hsic_file_reader = pd.read_csv(
        hsic_origin, 
        encoding='utf-8',
        index_col=0,
        sep ="\t",
        decimal=",",
        quotechar='"'
    )
    
    # Cargar palabras válidas
    words_file_reader = pd.read_csv(
        words_origin, 
        encoding='utf-8',
        skiprows = [1],
        index_col=0,
        sep ="\t",
        decimal=",",
        quotechar='"'
    )
    
    valid_words = words_file_reader.index.intersection(hsic_file_reader.index)
    valid_words = words_file_reader.loc[valid_words]

    
    # Cargar ciudades válidas
    cities_coordinates = pd.read_csv(
        cities_origin,
        encoding='utf-8',
        sep ="\t",
        decimal=",",
        quotechar='"',
        usecols=['ciudad', 'Latitud','Longitud'],
        index_col =0
    ).transpose()
      
    
    count=desde

    for count  in range(desde,hasta+1):
        start_time = time.time()
        word_data =  pd.DataFrame(valid_words.iloc[count]).transpose().sort_index()
        word = word_data.index[0]
        data = pd.concat([cities_coordinates,word_data]).transpose()
        word_vector = data.iloc[0:, 2:3].as_matrix()
        cities_vector = data.iloc[0:, 0:2].as_matrix()
        hsic, pvalue=HSIC_pval(cities_vector,word_vector,N_samp=500, kernelX="Gaussian", kernelY="Gaussian")
        basic_header = pd.DataFrame({
            'HSIC':pd.Series([], dtype='float'),
            'PValue':pd.Series([], dtype='float')
        })
        basic_header.loc['{}'.format(word),'HSIC']=hsic
        basic_header.loc['{}'.format(word),'PValue']=pvalue
        time_left = (time.time() - start_time)*(hasta+1-count)
        m, s = divmod(time_left, 60)
        h, m = divmod(m, 60)
        s=int(s)
        print ('\rCont:{}\t Word:{}\t\t PV:{} TimeLeft: {}:{}:{}\t '.format(count, word, pvalue, h, m, s),end='\t\t\r')
        if count == desde:
            escribir_cabezote_archivo(destino, basic_header)
        elif count <= hasta:
            escribir_datos_archivo(destino, basic_header)
        else:
            print("\rFrito el pollo")
        
        count = count + 1
         

In [5]:
hsicpvalue_to_file(HSIC_ORIGIN, WORDS_ORIGIN, CITIES_ORIGIN, DESTINO, 0, 20000)


IndexError: single positional indexer is out-of-bounds